In [27]:
!kaggle competitions download -c petfinder-pawpularity-score

petfinder-pawpularity-score.zip: Skipping, found more recently modified local copy (use --force to force download)


In [456]:
import pandas as pd
from os import walk
import glob
import numpy as np
from PIL import Image 
import cv2
import re
import torchvision.transforms as T
import torchvision.models as models
from torchvision.utils import make_grid
from torchvision.datasets import ImageFolder
from torch.utils.data import RandomSampler
from matplotlib import pyplot as plt
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
from torch.nn import functional as F
import torch

TRAIN_PATH = './input/train'
TEST_PATH = './input/test'


In [457]:
df_train = pd.read_csv(TRAIN_PATH+'.csv', index_col='Id')
df_train

,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur,Pawpularity
Id,,,,,,,,,,,,,
0007de18844b0dbbb5e1f607da0606e0,0,1,1,1,0,0,1,0,0,0,0,0,63
0009c66b9439883ba2750fb825e1d7db,0,1,1,0,0,0,0,0,0,0,0,0,42
0013fd999caf9a3efe1352ca1b0d937e,0,1,1,1,0,0,0,0,1,1,0,0,28
0018df346ac9c1d8413cfcc888ca8246,0,1,1,1,0,0,0,0,0,0,0,0,15
001dc955e10590d3ca4673f034feeef2,0,0,0,1,0,0,1,0,0,0,0,0,72
...,...,...,...,...,...,...,...,...,...,...,...,...,...
ffbfa0383c34dc513c95560d6e1fdb57,0,0,0,1,0,0,0,0,0,0,0,1,15
ffcc8532d76436fc79e50eb2e5238e45,0,1,1,1,0,0,0,0,0,0,0,0,70
ffdf2e8673a1da6fb80342fa3b119a20,0,1,1,1,0,0,0,0,1,1,0,0,20


In [458]:
images = []
for f in glob.iglob(TRAIN_PATH+'/*'):
    images.append(f)



In [459]:
input_train =[]
for i in images:
    regexp_1 = re.compile(r'\./input/[^/]*/(.*)\..*')
    re_match = regexp_1.match(i)
    input_train.append((i,tuple(df_train[df_train.index == re_match.groups()[0]].iloc[0].values)))

In [460]:
transform = T.Compose([
     T.Resize((256,256)), 
     T.ToTensor(),
     T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))

])

In [488]:
class CompleteDataset(Dataset):
    
    def __init__(self, imgs_list, transforms = None):
        
        super().__init__()
        self.imgs_list = imgs_list
        self.transforms = transforms
        
        
    def __getitem__(self, index):
    
        image_path = self.imgs_list[index][0]
        complentary_values = self.imgs_list[index][1][0:-1]
        popularity = self.imgs_list[index][1][-1]
        image = Image.open(image_path)
        #Reading image
        #Applying transforms on image
        if self.transforms:
            image = self.transforms(image)
        
        return image, torch.tensor(list(complentary_values)).float(),popularity
        
        
        
    def __len__(self):
        return len(self.imgs_list)

In [489]:
train_dataset = CompleteDataset(input_train, transform)

In [490]:
for i in range(0,5):
    
    print(train_dataset.__getitem__(i)[1])

tensor([0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0., 1., 1., 1., 0., 0., 0., 1., 1., 1., 0., 0.])
tensor([0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1.])
tensor([0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.])
tensor([0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.])


In [491]:
train_random_sampler = RandomSampler(train_dataset)
train_data_loader = DataLoader(
    dataset = train_dataset,
    batch_size = 16,
    sampler = train_random_sampler,
    num_workers = 4,
)

In [518]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Convolution 1
        self.cnn1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1)

        self.fc_cnn_1 = nn.Linear(16*254*254, 16)         
        self.fc_linear_1 = nn.Linear(12,16)
        self.fcout = nn.Linear(32, 1)    

    def forward(self, x):
        out_cnn = self.cnn1(x[0])
        
        out_cnn = out_cnn.view(out_cnn.size(0), -1)
        out_cnn = self.fc_cnn_1(out_cnn)
        out_linear = self.fc_linear_1(x[1])
        out = torch.cat((out_cnn, out_linear), dim=1)
        out = self.fcout(out)

        return out

In [519]:
model = Net()
model

Net(
  (cnn1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc_cnn_1): Linear(in_features=1032256, out_features=16, bias=True)
  (fc_linear_1): Linear(in_features=12, out_features=16, bias=True)
  (fcout): Linear(in_features=32, out_features=1, bias=True)
)

In [520]:
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
# lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 5, gamma = 0.75)
criterion = nn.MSELoss()

In [521]:
for i, (images,comp_values, labels) in enumerate(train_data_loader):

    optimizer.zero_grad()        
    preds = model([images, comp_values])
    loss = criterion(preds, labels.float())
    print(loss)

    loss.backward()
    optimizer.step()

